# Homework 4

Chris Dailey

christopher.dailey@gmail.com



In [1]:
#imports
import re
from mrjob.job import MRJob
from mrjob.step import MRJobStep

## HW 4.0
What is MrJob? How is it different to Hadoop MapReduce? 
What are the mappint_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?

## HW 4.1
What is serialization in the context of MrJob or Hadoop? 
When it used in these frameworks? 
What is the default serialization mode for input and outputs for MrJob? 


## HW 4.2

Preprocess the Microsoft log data into a format that can be analyzed in parallel

In [12]:
inputFile = open("anonymous-msweb.data", 'r')

outputFile = open("cleanedlog.txt", 'w')

current_user = ""

for line in inputFile.readlines():
    if line[0] in ["A","T","N", "I"]:
        outputFile.write(line)
    elif line[0] == "C":
        current_user = "C," + re.split(',', line)[2].strip()
    elif line[0] == "V":
        outputFile.write(line.strip() + "," + current_user + "\n")
        
outputFile.close()

## HW 4.3
Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).

In [79]:
%%writefile pagefrequency.py
import re
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.conf import combine_dicts

class MRFrequencyUtility(MRJob):
    
    def step_two_conf(self):
        #orig_jobconf = super(MRFrequencyUtility, self).jobconf()
        new_jobconf= {
            'mapred.output.key.comparator.class':'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options':'-k2,2n'
        }

        #return combine_dicts(orig_jobconf, new_jobconf) 
        return new_jobconf
    
    def mapper_get_pages(self, _, line):
        splits = [x.strip() for x in line.split(",")]
        if splits[0] =='V':
            yield (splits[1], 1)
                    
    def reducer_count_pages(self, page, counts):
        yield  page, sum(counts)
        
    def mapper_sort_counts(self, page, counts):
        yield count, page
    
    def reducer_sort_counts(self, page, counts):
        for count in counts:
            yield page, count
        
        
        

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_pages,
                    reducer=self.reducer_count_pages),
            MRStep(mapper=self.mapper_sort_counts,
                   reducer=self.reducer_sort_counts,
                   jobconf=self.step_two_conf())
        ]
    
if __name__ == '__main__':
    MRFrequencyUtility.run()

Overwriting pagefrequency.py


In [80]:
!hdfs dfs -rm /user/ubuntu/mrJobOutput/*
!hdfs dfs -rmdir /user/ubuntu/mrJobOutput
!python pagefrequency.py --jobconf -numReduceTasks=1 cleanedlog.txt --output-dir mrJobOutput -r hadoop \
--hadoop-home /home/ubuntu/hadoop-2.7.1/

16/02/07 20:39:21 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/_SUCCESS
16/02/07 20:39:21 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/part-00000
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/pagefrequency.ubuntu.20160207.203923.873940
writing wrapper script to /tmp/pagefrequency.ubuntu.20160207.203923.873940/setup-wrapper.sh
Using Hadoop version 2.7.1
Copying local files into hdfs:///user/ubuntu/tmp/mrjob/pagefrequency.ubuntu.20160207.203923.873940/files/

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-pr

In [69]:
!hdfs dfs -ls /user/ubuntu/mrJobOutput

Found 2 items
-rw-r--r--   1 ubuntu supergroup          0 2016-02-07 20:24 /user/ubuntu/mrJobOutput/_SUCCESS
-rw-r--r--   1 ubuntu supergroup          0 2016-02-07 20:24 /user/ubuntu/mrJobOutput/part-00000


In [71]:
!hdfs dfs -rm /user/ubuntu/mrJobOutput/*
!hdfs dfs -rmdir /user/ubuntu/mrJobOutput

16/02/07 20:26:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/_SUCCESS
16/02/07 20:26:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ubuntu/mrJobOutput/part-00000
